In [ ]:
%load_ext autoreload
%autoreload 2
from expressiveness_benchmark.types import Plan, Task, Language, SourceRange, Program
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
from interval import interval
from itertools import combinations
sns.set(style='whitegrid')

In [ ]:
programs = Program.load_all()
tasks = Task.load_all()

programs = programs[programs.plan != {}] # filter where plan hasn't been labeled
programs = programs[programs.task.isin(tasks.id.tolist())] # filter programs associated to a missing task
programs['category'] = programs.task.map(lambda t: tasks[tasks.id == t].category.iloc[0])

# Source metrics

In [ ]:
programs['nlines'] = programs.source.map(lambda s: len(s.split('\n')))
sns.boxplot(data=programs, x='language', y='nlines')

In [ ]:
programs['nchars'] = programs.source.map(len)
sns.boxplot(data=programs, x='language', y='nchars')

In [ ]:
from scipy.stats import zscore, rankdata

def bestworst(group, measure='nchars', score=rankdata):
    df = programs.groupby('task', as_index=False).apply(
        lambda rows: rows.assign(
            metric=score(rows[measure]))).reset_index()

    def f(df):
        df = df.groupby('category').metric.mean()
        return {'best': df.index[df.argmin()], 'worst': df.index[df.argmax()]}

    return df.groupby('language').apply(f)

In [ ]:
def inv_zscore(x):
    return -zscore(x)

for group in ['category', 'task']:
    for score in [rankdata, inv_zscore]:
        print(f'GROUP: {group}, SCORE: {score.__name__}')
        print(bestworst(group, score=score))
        print()

In [ ]:
bestworst('category', score=rankdata)

# Plan metrics

In [ ]:
def coord_to_1d(lines, line, column):
    return sum([len(l) for l in lines[:line]]) + column        
    
def plan_overlap(row):
    lines = row.source.split('\n')
    plan_df = pd.DataFrame([{'key': k, **v} for k, vs in row.plan.items() for v in vs])
    plan_df['byte_start'] = plan_df.apply(lambda row: coord_to_1d(lines, row.line, row.start), axis=1)
    plan_df['byte_end'] = plan_df.apply(lambda row: coord_to_1d(lines, row.line, row.end), axis=1)

    full_intervals = plan_df.groupby('key').apply(
        lambda rows: interval[rows.byte_start.min(), rows.byte_end.max()])
    all_pairs = combinations(full_intervals.tolist(), r=2)
    return sum([1 if l & r else 0 for (l, r) in all_pairs])

programs['plan_overlap'] = programs.apply(plan_overlap, axis=1)
order = programs.groupby('language').plan_overlap.mean().sort_values().index.values
g = sns.FacetGrid(data=programs, row='language', aspect=2, height=2, row_order=order)
g.map(sns.distplot, 'plan_overlap', kde=False, bins=np.arange(0, programs.plan_overlap.max()+1))
for i, ax in enumerate(g.axes.flatten()):
    ax.axvline(programs[programs.language == order[i]].plan_overlap.mean(), color='r')
    ax.tick_params(labelbottom=True)    
plt.tight_layout()